In [4]:
import numpy as np
from qibo.models import VQE
from qibochem.driver.molecule import Molecule
from qibochem.ansatz import hf_circuit, ucc_circuit
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import I, X, Y, Z
from qibo import Circuit, gates
from qibo.optimizers import optimize
import time


def perform_shadow_snapshots(circuit, shadow_size, num_qubits):

    total_time = time.time()
    times = np.zeros(5)
    # Maps the Pauli measurements to the indeces 0, 1, 2 = X, Y, Z
    Pauli = [gates.X, gates.Y, gates.Z]

    # Measurement basis are chosen at random
    measurement_basis = np.random.randint(0, 3, size = (shadow_size, num_qubits))
    outcomes = np.zeros((shadow_size, num_qubits))

    for i in range(shadow_size):
        # In each snapshot, we perform a measurement in a randomly chosen basis
        c = circuit.copy()
        c.add(gates.M(*range(num_qubits), basis = [Pauli[measurement_basis[i][j]] for j in range(num_qubits)]))
        outcomes[i] = c(nshots = 1).samples(binary = True)[0]

    # Returns both the measurement outcomes and the basis the measurements were performed in
    return (outcomes, measurement_basis)

def calculate_classical_shadow(circuit, shadow_size, num_qubits):

    # Performs the measurements
    outcomes, measurement_basis = perform_shadow_snapshots(circuit, shadow_size, num_qubits)
    

    # Possible states in matrix form
    states = [np.array([[1, 0],[0, 0]]), np.array([[0, 0], [0, 1]])]
    # Matrices to invert the implicit unitary operations performed when measuring in different basis
    unitaries = [gates.H(0).matrix(), gates.H(0).matrix()@gates.S(0).dagger().matrix(), gates.I(0).matrix()]
    shadow = np.full(shadow_size, [1], dtype = np.ndarray)

    # Computes the classical shadows for each snapshots
    for i in range(shadow_size):
        for j in range(num_qubits):
            state = states[int(outcomes[i][j])]
            U = unitaries[measurement_basis[i][j]]
            local_rho = 3*(U.conj().T @ state @ U) - np.eye(2)
            shadow[i] = np.kron(shadow[i], local_rho)
            


    return shadow       

def estimate_energy(circuit, hamiltonian, N, K, num_qubits):
    
    
    shadow = calculate_classical_shadow(circuit, N, num_qubits)
    chunk_size = N//K
    chunk_rhos = np.zeros(K, dtype = np.ndarray)
    for k in range(K):
        for j in range (k*chunk_size, (k + 1)*chunk_size):
            chunk_rhos[k] = chunk_rhos[k] + shadow[j]
        chunk_rhos[k] = chunk_rhos[k]/chunk_size
    
    return np.median([(hamiltonian@chunk_rhos[k]).trace() for k in range(K)])



def cost(theta, qubits, hamiltonian):
    double_excitation = np.eye(16)
    double_excitation[3][3] = np.cos(theta/2)[0]
    double_excitation[12][12] = double_excitation[3][3]
    double_excitation[3][12] = np.sin(theta/2)[0]
    double_excitation[12][3] = -double_excitation[3][12]
    c = Circuit(qubits)
    c.add(gates.X(0))
    c.add(gates.X(1))
    c.add(gates.Unitary(double_excitation, 0, 1, 2, 3))
    e = estimate_energy(c, hamiltonian, 1000, 10, qubits)
    return e


mol = Molecule([('H', (0.0, 0.0, -0.3707)), ('H', (0.0, 0.0, 0.3707))])
mol.run_pyscf()
ham = mol.hamiltonian().matrix
qubits = mol.nso
initial_theta = np.pi

print(mol.hamiltonian().eigenvalues())

optimize(cost, initial_theta, args = (qubits, ham), tol = 1e-6)


converged SCF energy = -1.11668438708534


[Qibo 0.2.9|INFO|2024-08-08 11:25:41]: Using numpy backend on /CPU:0
[Qibo 0.2.9|WARNING|2024-08-08 11:25:41]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.9|WARNING|2024-08-08 11:25:41]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


[-1.13727017 -0.53870958 -0.53870958 -0.53247901 -0.53247901 -0.53247901
 -0.44698572 -0.44698572 -0.16990139  0.23780528  0.23780528  0.35243414
  0.35243414  0.47983612  0.71375399  0.92010672]


/tmp/ipykernel_746/1956474050.py:73: ComplexWarning: Casting complex values to real discards the imaginary part
  double_excitation[3][3] = np.cos(theta/2)[0]
/tmp/ipykernel_746/1956474050.py:75: ComplexWarning: Casting complex values to real discards the imaginary part
  double_excitation[3][12] = np.sin(theta/2)[0]
/home/alvarodil/anaconda3/envs/shadow/lib/python3.10/site-packages/scipy/optimize/_optimize.py:3542: ComplexWarning: Casting complex values to real discards the imaginary part
  direc[-1] = direc1


85.73129606246948


In [43]:
import numpy as np
from qibo.models import VQE
from qibochem.driver.molecule import Molecule
from qibochem.ansatz import hf_circuit, ucc_circuit
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import I, X, Y, Z
from qibo import Circuit, gates
from qibo.optimizers import optimize
from scipy.optimize import Bounds
import time


def perform_shadow_snapshots(circuit, shadow_size, num_qubits):

    total_time = time.time()
    times = np.zeros(5)
    # Maps the Pauli measurements to the indeces 0, 1, 2 = X, Y, Z
    Pauli = [gates.X, gates.Y, gates.Z]

    # Measurement basis are chosen at random
    measurement_basis = np.random.randint(0, 3, size = (shadow_size, num_qubits))
    outcomes = np.zeros((shadow_size, num_qubits))

    for i in range(shadow_size):
        # In each snapshot, we perform a measurement in a randomly chosen basis
        c = circuit.copy()
        c.add(gates.M(*range(num_qubits), basis = [Pauli[measurement_basis[i][j]] for j in range(num_qubits)]))
        outcomes[i] = c(nshots = 1).samples(binary = True)[0]

    # Returns both the measurement outcomes and the basis the measurements were performed in
    return (outcomes, measurement_basis)

def calculate_classical_shadow(circuit, shadow_size, num_qubits):

    # Performs the measurements
    outcomes, measurement_basis = perform_shadow_snapshots(circuit, shadow_size, num_qubits)
    

    # Possible states in matrix form
    states = [np.array([[1, 0],[0, 0]]), np.array([[0, 0], [0, 1]])]
    # Matrices to invert the implicit unitary operations performed when measuring in different basis
    unitaries = [gates.H(0).matrix(), gates.H(0).matrix()@gates.S(0).dagger().matrix(), gates.I(0).matrix()]
    shadow = np.full(shadow_size, [1], dtype = np.ndarray)

    # Computes the classical shadows for each snapshots
    for i in range(shadow_size):
        for j in range(num_qubits):
            state = states[int(outcomes[i][j])]
            U = unitaries[measurement_basis[i][j]]
            local_rho = 3*(U.conj().T @ state @ U) - np.eye(2)
            shadow[i] = np.kron(shadow[i], local_rho)
            


    return shadow       

def estimate_energy(circuit, hamiltonian, N, K, num_qubits):
    
    
    shadow = calculate_classical_shadow(circuit, N, num_qubits)
    chunk_size = N//K
    chunk_rhos = np.zeros(K, dtype = np.ndarray)
    for k in range(K):
        for j in range (k*chunk_size, (k + 1)*chunk_size):
            chunk_rhos[k] = chunk_rhos[k] + shadow[j]
        chunk_rhos[k] = chunk_rhos[k]/chunk_size
    
    return np.median([(hamiltonian@chunk_rhos[k]).trace() for k in range(K)])



def cost(theta, num_qubits, num_layers, rho):
    
    c = Circuit(num_qubits)
    for j in range(num_layers):
        for i in range(num_qubits):
            c.add(gates.RX(i, theta[j*num_qubits + i]))
            c.add(gates.CNOT(i, (i + 1)%num_qubits))
    guess_rho = c().state() 
    return 1 - abs(estimate_energy(c, rho, 1000, 10, num_qubits))
    

num_qubits = 2
num_layers = 10
c = Circuit(num_qubits, density_matrix = True)
rng = np.random.default_rng()
theta = rng.uniform(0,2*np.pi, num_qubits*num_layers)
initial_theta = np.zeros(num_qubits*num_layers)
for j in range(num_layers):
    for i in range(num_qubits):
        c.add(gates.RX(i, theta[j*num_qubits + i]))
        c.add(gates.CNOT(i, (i + 1)%num_qubits))
rho = c().state()

print(theta)


optimize(cost, initial_theta, args = (num_qubits, num_layers, rho), bounds = Bounds(np.zeros(num_qubits*num_layers), np.full(num_qubits*num_layers, 2*np.pi)))


[1.69456148 1.18097169 2.48253595 2.97949114 1.37723364 3.48766704
 3.80848844 2.00073083 0.89189821 5.13615752 2.12015594 0.51021749
 0.24641782 4.61839659 1.06932738 4.24451864 4.6655128  2.99056743
 2.91629794 5.79307899]
aaa
0.7038481030264871
(0.7746027044424285-6.938893903907228e-18j)
aaa
0.9294114032385884
(0.9867969518105046+1.734723475976807e-17j)
aaa
0.8107197523107919
(0.8341809761709259-3.8163916471489756e-17j)
aaa
0.8022668551663255
(0.7334443553060584-5.204170427930421e-17j)
aaa
0.7280870758422479
(0.7590900370261435-5.204170427930421e-17j)
aaa
0.7675186134338884
(0.7746092618143648-5.204170427930421e-17j)
aaa
0.8278578182453957
(0.7546787893890003-5.204170427930421e-17j)
aaa
0.7815252535134938
(0.759093392785798-5.204170427930421e-17j)
aaa
0.7736326437940475
(0.7573823401418747-4.85722573273506e-17j)
aaa
0.7183986745394361
(0.7584344793959893-5.204170427930421e-17j)
aaa
0.7041442446034014
(0.7580313421894305-4.85722573273506e-17j)
aaa
0.7973465074913849
(0.75778296470773

KeyboardInterrupt: 